# 7. 앙상블 학습과 랜덤 포레스트
여러 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있을 것이다. 이를 **앙상블 학습**이라고 한다.
특히 결정 트리의 앙상블을 **랜덤 포레스트**라고 한다.

## 7.1 투표 기반 분류기
정확도가 80%인 분류기 여러 개를 훈련시켰다고 가정하자.
이를 더 좋은 분류기로 만드는 가장 간단한 방법은 각 분류기의 예측을 모아서 가장 많이 선택된 클래스로 예측하는 것이다.
이렇게 다수결로 정해지는 분류기를 **직접 투표 분류기**라고 한다.

직접 투표 분류기가 분류기 집단 내에서 가장 성능이 좋은 분류기보다 더 정확도가 높을 때가 많다.

In [1]:
# 데이터셋 만들기
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("rf", rnd_clf), ("svc", svm_clf)],
    voting="hard"
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.896
VotingClassifier 0.896


투표 기반 분류기가 분류기의 구성원들보다 성능이 더 좋은 것을 확인할 수 있다.

<u>모든 분류기가 클래스의 확률을 예측할 수 있으면</u> 개별 분류기의 예측을 평균 내어 가장 높은 클래스를 예측할 수 있다. 이를 **간접 투표**라고 한다.
이 방식은 느리지만 직접 투표 방식보다 성능이 높다.

## 7.2 배깅과 페이스팅
투표 분류기와는 반대로 같은 알고리즘을 사용해서 훈련 세트의 서로 다른 부분을 학습하도록 할 수 있다.
이 때 훈련 세트에 중복을 허용하는 방식을 **배깅**, 중복을 허용하지 않는 방식을 **페이스팅**이라고 한다.

모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새오룬 샘플에 대한 예측을 만든다.
수집 함수는 일반적으로

- 분류일 때는 통계적 최빈값
- 회귀일 때는 평균

### 7.2.1 사이킷런의 배깅과 페이스팅
사이킷런에서는 배깅과 페이스팅을 위한 간편한 API로 구성된 `BaggingClassifier`와 `BaggingRegressor`를 제공한다.
각 분류기가 훈련 세트에서 중복을 허용하여 무작위로 선택해 훈련(배깅)하고, `bootstrap=False` 옵션을 적용하면 페이스팅을 사용할 수 있다.

In [4]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

앙상블을 사용하면 오차 수가 거의 비슷하지만, 결정 경계는 덜 불규칙적이게 된다.

부트스트래핑은 각 예측기가 학습하는 서브셋의 다양성을 증가시키므로 배깅이 페이스팅보다 편향이 더 높다.
하지만 서브셋의 다양성을 증가시키는 것은 예측기의 상관관계를 줄이기 때문에 앙상블의 분산을 감소시킨다.

전반적으로 배깅이 페이스팅보다 더 나은 모델을 만들어내지만 여유가 있다면 교차 검증으로 더 나은 쪽을 선택하는 것이 좋다.

### 7.2.2 oob 평가
배깅을 사용하면 어떤 샘플은 여러 번 샘플링되고, 어떤 것은 전혀 선택되지 않을 수 있다.
이 때 선택되지 않은 샘플을 oob 샘플이라고 부른다.
사이킷런에서 `BaggingClassifier`를 수행할 때 `oob_score=True`로 지정하면 훈련이 끝날 때 자동으로 oob 평가를 수행한다.

In [5]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.896

In [6]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.904

oob score와 테스트 세트에서의 정확도가 어느 정도 비슷하다.

## 7.3 랜덤 패치와 랜덤 서브스페이스
`BaggingClassifier`는 특성 샘플링도 지원한다. 각 예측기는 무작위로 선택된 입력 특성의 일부분으로 훈련하게 된다.

이 기법은 이미지와 같은 고차원의 데이터셋을 사용할 때 유용하다.
훈련 특성과 샘플을 모두 샘플링하는 것을 **랜덤 패치 방식**이라고 하고,
훈련 샘플을 모두 사용하고 특성은 샘플링하는 것을 **랜덤 서브스페이스 방식**이라고 한다.

## 7.4 랜덤 포레스트
**랜덤 포레스트**는 배깅(또는 페이스팅)을 적용한 결정 트리의 앙상블이다.

In [7]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신,
선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입한다. 이는 트리를 더 다양하게 만들고,
분산을 낮추어 전체적으로 좋은 모델을 만들어낸다.

### 7.4.1 엑스트라 트리
극단적으로 무작위한 트리의 랜덤 포레스트를 **익스트림 랜덤 트리**(줄여서 **엑스트라 트리**)라고 한다.

### 7.4.2 특성 중요도
랜덤 포레스트의 또 다른 장점은 특성의 상대적 중요도를 측정하기 쉽다는 것이다.
사이킷런에서는 어떤 특성을 사용하는 노드가 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정한다.

In [8]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.10104428918635133
sepal width (cm) 0.022406321014287706
petal length (cm) 0.4287291908830082
petal width (cm) 0.44782019891635266


위 결과는 붓꽃 데이터셋에 `RandomForestClassifier`를 적용하고 각 특성의 중요도를 출력한 것이다.
결과를 봤을 때 꽃잎의 길이(약 42%)와 너비(44%)가 중요한 특성이고,
꽃받침의 길이(10%)와 너비(2%)는 그보다는 덜 중요해 보인다.